### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [2]:
#Imports the daily TJ06 (options) & TX06 (forwards) and updates FX_Derivatives_All Database 
#which is located in the Financial_Database directory located within the Treasury drive

import pandas as pd
import csv
import datetime

#this function module returns the delta days for file date (ie. If today is Monday, we need Friday's date
#for the SAP file)
def whatday():
    wkday = datetime.datetime.today().weekday()
    if wkday == 0:    
        return 3
    else:
        return 1

#this function module converts floats to strings.
def change2str(df, column):
    df[column] = df[column].astype(int)
    df[column] = df[column].astype(str)
    
    
#Putting into a variable date to be used with the file names
today = datetime.date.today()
prevday = today - datetime.timedelta(days=whatday())
prevday = prevday.strftime('%m%d%Y')

#Build file paths to get the daily TJ06 (options) and TX06 (forwards) files. 
dir_path = r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256'
file1_name = '\TX06_Extract' + prevday + '.txt'
file2_name = '\TJ06_Extract' + prevday + '.txt'
file_path1 = dir_path + file1_name
file_path2 = dir_path + file2_name

#Formating the header for daily TJ06 (options) and TX06 (forwards) files.
df1 = pd.read_csv(file_path1, delimiter = '|', skiprows=[0,2], header =[0])
df2 = pd.read_csv(file_path2, delimiter = '|', header =[0])

#Aligned header to match data columns for TJ06 (Option) report and removed blank spaces on header fields.
names = list(df2.columns.values)
newname = []
for name in names:
    newname.append(name.strip())
newname.append(newname.pop(0))
df2.columns = newname
df2.reset_index()


df2.insert(0,"Entity", 'ADIR')
df2.dropna(subset = ["Transactn"], inplace=True)

df2.reset_index(drop=True, inplace=True)

df2.dropna(axis=1, how='all', inplace=True)

#create dictionary of counter party names and their associated numbers
counter_party_dicts = {'61':'JPM',
                       '2':'BOA',
                       '2081':'USB',
                       '2121':'WFB',
                       '2101':'HSBC',
                       '12600': 'SG',
                       '1200000141': 'BOA',
                       '2131':'MSC',
                       '3':'UBS',
                       '5':'GS',
                       '72':'CITI',
                       '10020':'RBS'}

#This changes format by using the change2str function.
change2str(df2, 'Counterparty')
change2str(df2, 'Transactn')   

#This converts the SAP BP code to a counterparty name.
i=0
for counterparties in df2.loc[:,'Counterparty']:
    
    df2.loc[i,'Counterparty'] = counter_party_dicts[counterparties]
    i+=1

#Creating dictionary to add currency pair
currencypair_dict = {'EUR':'EURUSD',
                    'GBP':'GBPUSD',
                    'AUD':'AUDUSD',
                    'JPY':'USDJPY',
                    'INR':'USDINR'}    

#Add additional columns
df2.insert(0, 'ID',"ADIR" + df2['Transactn'])

#Removing unwanted columns
df2.drop(columns=["Type", "Exercise", "Settlement", "Premium Curr", "Transactn"], inplace= True)



#Add additional columns
df2['Currency_Pair'], df2['Buy_Currency'], df2['Sell_Currency'], df2['Buy_Amount'] = ["", "USD", "", 0]

#Split underlying column into sell amount and sell currency amount.
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    
    df2.loc[index,'Sell_Currency'] = Curr
    df2.loc[index,'Currency_Pair'] = currencypair_dict[Curr]
    
#Adding Buy Amount buy multiplying the strike and underlying. Also trancating the Buy_Amount to two decimals
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    Amt = float(Amt.replace(',','').strip('-'))
    df2.loc[index,'Buy_Amount'] = Amt*df2.loc[index,'Strike']
    df2.loc[index,'Underlying'] = Amt*-1
    
#Rename columns to match FX Derivatives all
df2.rename(columns = {"Premium Amt":"Premium", 
                      "Term Start":"Trade_Date", 
                      "Due Date":"Expiry_Date", 
                      "Strike":"Contract_Rate",
                      "Underlying":"Sell_Amount"},inplace=True)


In [4]:
df2

#type(df2.loc[0,'Amt'])

#Move Counterparty column to third column
#df = df[['a', 'y', 'b', 'x']]

,ID,Entity,Trade_Date,Expiry_Date,Sell_Amount,Contract_Rate,Counterparty,Premium,Currency_Pair,Buy_Currency,Sell_Currency,Buy_Amount
0,ADIR7902,ADIR,03/24/2020,10/08/2020,-3e+07,1.0865,BOA,"909,013.00",EURUSD,USD,EUR,3.259500e+07
1,ADIR7930,ADIR,04/07/2020,10/15/2020,-3.2e+07,1.0950,WFB,"832,800.00",EURUSD,USD,EUR,3.504000e+07
2,ADIR7931,ADIR,04/07/2020,10/15/2020,-2e+07,1.2350,WFB,"808,500.00",GBPUSD,USD,GBP,2.470000e+07
3,ADIR7932,ADIR,04/07/2020,10/15/2020,-2e+09,108.3500,JPM,"462,574.99",USDJPY,USD,JPY,2.167000e+11
4,ADIR7985,ADIR,05/05/2020,10/29/2020,-1.5e+07,0.6450,BOA,"333,672.00",AUDUSD,USD,AUD,9.675000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
61,ADIR8275,ADIR,09/29/2020,04/22/2021,-1.5e+07,0.7125,BOA,"340,977.00",AUDUSD,USD,AUD,1.068750e+07
62,ADIR8276,ADIR,09/29/2020,04/22/2021,-3.7e+07,1.1792,WFB,"921,300.00",EURUSD,USD,EUR,4.363040e+07
63,ADIR8277,ADIR,09/29/2020,04/22/2021,-1.2e+07,1.2852,HSBC,"493,057.39",GBPUSD,USD,GBP,1.542240e+07
64,ADIR8278,ADIR,09/29/2020,04/22/2021,-3.3e+09,105.3300,BOA,"729,338.00",USDJPY,USD,JPY,3.475890e+11
